In [56]:
import json
import os
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to
[nltk_data]     /datasets/home/22/022/cahou/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /datasets/home/22/022/cahou/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
lv = pd.read_csv("../data/raw/Las_Vegas_reviews.csv")

In [18]:
# with open('../data/autophrase_in/reviews.txt', 'w') as f:
#     f.write('test')
# with open('../data/autophrase_out/segmentation.txt', 'r') as f:
with open('../../DSC180A-Project/data/out/segmentation.txt', 'r') as f:
# with open('../../DSC180A-Project/data/in/yelp_reviews.txt', 'r') as f:
# with open('../../DSC180A-Project/data/in/yelp_reviews1.txt', 'r') as f:
    reviews_string = f.read()
    reviews_list = a.split("\n.\n")

In [6]:
sid = SentimentIntensityAnalyzer()

In [57]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer.tokenize(reviews_list[0])

['So why would I be giving a <phrase>Fast Food chain</phrase> location a 5 <phrase>star</phrase> rating?',
 "For some reason, THIS <phrase>Carl's Jr</phrase>.",
 'is one in a million.',
 "Yes, there is an In-and-Out across the street but this <phrase>Carl's Jr</phrase> is everything In-and-Out can't be.",
 'First of all, THEIR SHAKES!!!',
 "<phrase>Carl's Jr</phrase>.",
 "shakes are <phrase>hand scooped</phrase>, and mixed upon <phrase>order</phrase> unlike they're competitors across the street.",
 "The <phrase>chocolate</phrase> shake IS the best I've EVER had from a <phrase>drive thru</phrase>.",
 "Point for <phrase>Carl's Jr</phrase>.",
 'Second, the <phrase>hand battered</phrase> <phrase>chicken</phrase> <phrase>strips</phrase> were literally <phrase>hand battered</phrase>, i <phrase>watch</phrase> the guy through the service window dip and fry.',
 "The <phrase>single</phrase> best <phrase>chicken</phrase> <phrase>strips</phrase> i've ever had (but they may need new <phrase>sauces<

In [83]:
scores = []
seen_sentences = []
index = []
for i, review in enumerate(reviews_list[0:1]):
    sentences = tokenizer.tokenize(review)
    for sentence in sentences:
        index.append(i)
        seen_sentences.append(sentence)
        ss_intermediate = sid.polarity_scores(sentence)
        scores.append(ss_intermediate)

In [84]:
df = pd.DataFrame({'index': i, 'sentence': sentences}).join(pd.DataFrame(scores))

In [85]:
df

,index,sentence,neg,neu,pos,compound
0,0,So why would I be giving a <phrase>Fast Food c...,0.000,0.806,0.194,0.3400
1,0,"For some reason, THIS <phrase>Carl's Jr</phrase>.",0.000,1.000,0.000,0.0000
2,0,is one in a million.,0.000,1.000,0.000,0.0000
3,0,"Yes, there is an In-and-Out across the street ...",0.000,0.896,0.104,0.2144
4,0,"First of all, THEIR SHAKES!!!",0.000,1.000,0.000,0.0000
5,0,<phrase>Carl's Jr</phrase>.,0.000,1.000,0.000,0.0000
6,0,"shakes are <phrase>hand scooped</phrase>, and ...",0.000,1.000,0.000,0.0000
7,0,The <phrase>chocolate</phrase> shake IS the be...,0.107,0.629,0.264,0.5423
8,0,Point for <phrase>Carl's Jr</phrase>.,0.000,1.000,0.000,0.0000
9,0,"Second, the <phrase>hand battered</phrase> <ph...",0.000,1.000,0.000,0.0000
